In [6]:
import numpy as np
import cv2
import time

In [2]:
# Helper functions for finite differences
# calculates the gradients of an image x 
def D(x):
    s = x.shape
    grad = np.zeros([s[0], s[1],2])
    grad[:,:,0] = x - np.roll(x, (-1, 0), axis=(0, 1))
    grad[s[0]-1, :, 0] = 0
    grad[:, :, 1] = x - np.roll(x, (0, -1), axis=(0, 1))
    grad[:, s[1]-1, 1] = 0
    return grad

In [3]:
# calculates the adjoint operator of D
def DT(grad):
    n1, n2, _ = grad.shape
    shift = np.roll(grad[:, :, 0], (1, 0), axis=(0, 1))
    div1 = grad[:, :, 0] - shift
    div1[0, :] = grad[0, :, 0]
    div1[n1-1, :] = -shift[n1-1, :]

    shift = np.roll(grad[:, :, 1], (0, 1), axis=(0, 1))
    div2 = grad[:, :, 1] - shift
    div2[:, 0] = grad[:, 0, 1]
    div2[:, n2-1] = -shift[:, n2-1]

    div = div1 + div2 
    return div

In [4]:
def FGP_gray2d(y, lambd, n_iters, tv_type='anisotropic', \
               ground_truth = None, eps = 1e-3):
    # Main loop
    n1, n2 = y.shape
    grad_next = np.zeros((n1, n2, 2))
    grad_prev = np.zeros((n1, n2, 2))
    u = np.zeros((n1, n2, 2))
    t_prev = 1
    tic = time.perf_counter()
    if ground_truth == None:
        # if the original is not given
        for i in range(n_iters):
            grad_next = u + 1 / (8 * lambd) * D(y - lambd * DT(u))
            deno = np.zeros((n1, n2, 2))
            
            if tv_type == 'anisotropic':
                deno[:, :, 0] = np.maximum(1, np.abs(grad_next[:, :, 0]))
                deno[:, :, 1] = np.maximum(1, np.abs(grad_next[:, :, 1]))
            else:
                deno[:, :, 0] = np.maximum(1, np.sqrt(grad_next[:, :, 0]**2 + grad_next[:, :, 1]**2))
                deno[:, :, 1] = deno[:, :, 0]
            
            grad_next /= deno
            t_next = (1 + np.sqrt(1 + 4 * t_prev**2)) / 2
            u = grad_next + (t_prev - 1) / t_next * (grad_next - grad_prev)
            grad_prev = grad_next
            t_prev = t_next
    else:
        k = 1
        while np.linalg.norm(ground_truth, x)>eps:
            grad_next = u + 1 / (8 * lambd) * D(y - lambd * DT(u))
            deno = np.zeros((n1, n2, 2))
            
            if tv_type == 'anisotropic':
                deno[:, :, 0] = np.maximum(1, np.abs(grad_next[:, :, 0]))
                deno[:, :, 1] = np.maximum(1, np.abs(grad_next[:, :, 1]))
            else:
                deno[:, :, 0] = np.maximum(1, np.sqrt(grad_next[:, :, 0]**2 + grad_next[:, :, 1]**2))
                deno[:, :, 1] = deno[:, :, 0]
            
            grad_next /= deno
            t_next = (1 + np.sqrt(1 + 4 * t_prev**2)) / 2
            u = grad_next + (t_prev - 1) / t_next * (grad_next - grad_prev)
            grad_prev = grad_next
            t_prev = t_next 
            k+=1
            if k > n_iters: 
                break
    x = y - lambd * DT(grad_next)
    toc = time.perf_counter()
    runtime = toc - tic 

    return x, runtime

In [5]:
if __name__ == "__main__":
    fileName = "input.png"
    N = 100
    lambd = 40

    u = cv2.imread("input.png", cv2.IMREAD_GRAYSCALE)
    #print(u)
    u = u.astype(np.float32)#/ 255.0

    result, runtime = FGP_gray2d(u, lambd, N, tv_type='anisotropic', \
               ground_truth = None, eps = 1e-3)
    #result *= 255
    result = result.astype(np.uint8)
    cv2.imwrite("output_FGP.png", result)